In [44]:
# import pandas
import pandas as pd
# load data
dataset = pd.read_csv("C:\\Users\\Admin\\Documents\\data_mining\\DataSetA.csv")
# size of data
dataset.shape

(8, 7)

In [45]:
dataset.head()

,Rice,Noodle,Fish,Meal,Vegetable,Soup,Salad
0,Pizza,Cake,Rice,Candy,Salad,Rice,Milk
1,Rice,Milk,Fish,Candy,NaN,NaN,NaN
2,Rice,Noodle,Rice,Meal,Rice,Rice,Candy
3,Rice,Noodle,Meal,Soup,NaN,NaN,NaN
4,NaN,NaN,Rice,Rice,Noodle,Milk,NaN


In [46]:
# import numpy
import numpy as np
# Tạo 1 mảng lưu tất cả các dữ liệu trong dataset
transaction = []
for i in range(0, dataset.shape[0]):
    for j in range(0, dataset.shape[1]):
        transaction.append(dataset.values[i,j])
# chuyển thành mảng numpy
transaction = np.array(transaction)
print(transaction)

['Pizza' 'Cake' 'Rice' 'Candy' 'Salad' 'Rice' 'Milk' 'Rice' 'Milk' 'Fish'
 'Candy' 'nan' 'nan' 'nan' 'Rice' 'Noodle' 'Rice' 'Meal' 'Rice' 'Rice'
 'Candy' 'Rice' 'Noodle' 'Meal' 'Soup' 'nan' 'nan' 'nan' 'nan' 'nan'
 'Rice' 'Rice' 'Noodle' 'Milk' 'nan' 'nan' 'Pizza' 'Pizza' 'Rice' 'Rice'
 'nan' 'nan' 'nan' 'nan' 'nan' 'nan' 'Noodle' 'Noodle' 'Noodle' 'nan'
 'nan' 'Rice' 'Rice' 'Rice' 'nan' 'nan']


In [47]:
#  Chuyển đổi mảng numpy thành dataframe
df = pd.DataFrame(transaction, columns=["items"]) 
# Tạo biến đếm, đếm tần suất xuất hiện của các item
df["incident_count"] = 1 
#  Xóa các item có kiểu dữ liệu null 
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
# tạo df đếm số lượng xuất hiện của các item
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
#  lấy top 5 item xuất hiện nhiều nhất
df_table.head(5).style.background_gradient(cmap='Blues')

,items,incident_count
0,Rice,15
1,Noodle,6
2,Candy,3
3,Milk,3
4,Pizza,3


Vẽ biểu đồ thể hiện tần suất xuất hiện của các item

In [48]:
# import 
import plotly.express as px
# to have a same origin
df_table["all"] = "Top 10 items" 
# creating tree map using plotly
fig = px.treemap(df_table.head(10), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(10), hover_data=['items'],
                  color_continuous_scale='Reds',
                )
# ploting the treemap
fig.show()

In [49]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
transaction = []
for i in range(dataset.shape[0]):
    transaction.append([str(dataset.values[i,j]) for j in range(dataset.shape[1])])
# creating the numpy array of the transactions
transaction = np.array(transaction)
# importing the required module
from mlxtend.preprocessing import TransactionEncoder
# initializing the transactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
# dataset after encoded
dataset.head()

,Cake,Candy,Fish,Meal,Milk,Noodle,Pizza,Rice,Salad,Soup,nan
0,True,True,False,False,True,False,True,True,True,False,False
1,False,True,True,False,True,False,False,True,False,False,True
2,False,True,False,True,False,True,False,True,False,False,False
3,False,False,False,True,False,True,False,True,False,True,True
4,False,False,False,False,True,True,False,True,False,False,True


In [50]:
# Chọn item
first10 = df_table["items"].values 
# Extract item
dataset = dataset.loc[:,first10] 
# size of dataset
dataset.shape

(8, 10)

Tìm item header table, tùy chỉnh min_support

In [51]:
#Importing Libraries
from mlxtend.frequent_patterns import fpgrowth
#running the fpgrowth algorithm
res=fpgrowth(dataset,min_support=0.05, use_colnames=True)
# printing top 10
res.head(100)

,support,itemsets
0,0.875,(Rice)
1,0.375,(Milk)
2,0.375,(Candy)
3,0.250,(Pizza)
4,0.125,(Salad)
...,...,...
88,0.125,"(Rice, Soup)"
89,0.125,"(Meal, Soup, Noodle)"
90,0.125,"(Meal, Rice, Soup)"
91,0.125,"(Rice, Soup, Noodle)"


Xác định các item set phổ biến

In [52]:
# importing required module
from mlxtend.frequent_patterns import association_rules
# creating asssociation rules
res=association_rules(res, metric="lift", min_threshold=1)
# printing association rules
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Rice),(Milk),0.875,0.375,0.375,0.428571,1.142857,0.046875,1.093750,1.000000
1,(Milk),(Rice),0.375,0.875,0.375,1.000000,1.142857,0.046875,inf,0.200000
2,"(Milk, Noodle)",(Rice),0.125,0.875,0.125,1.000000,1.142857,0.015625,inf,0.142857
3,(Rice),"(Milk, Noodle)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
4,(Rice),(Candy),0.875,0.375,0.375,0.428571,1.142857,0.046875,1.093750,1.000000
...,...,...,...,...,...,...,...,...,...,...
715,"(Soup, Noodle)","(Meal, Rice)",0.125,0.250,0.125,1.000000,4.000000,0.093750,inf,0.857143
716,(Meal),"(Rice, Soup, Noodle)",0.250,0.125,0.125,0.500000,4.000000,0.093750,1.750000,1.000000
717,(Rice),"(Meal, Soup, Noodle)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
718,(Soup),"(Meal, Rice, Noodle)",0.125,0.250,0.125,1.000000,4.000000,0.093750,inf,0.857143


Sắp xếp lại các item set phổ biến

In [53]:
# Sort values based on confidence
res.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
282,"(Rice, Cake, Salad)",(Pizza),0.125,0.250,0.125,1.000000,4.000000,0.093750,inf,0.857143
1,(Milk),(Rice),0.375,0.875,0.375,1.000000,1.142857,0.046875,inf,0.200000
591,"(Milk, Pizza)","(Cake, Rice, Candy, Salad)",0.125,0.125,0.125,1.000000,8.000000,0.109375,inf,1.000000
376,(Cake),"(Rice, Milk, Pizza)",0.125,0.125,0.125,1.000000,8.000000,0.109375,inf,1.000000
375,"(Milk, Pizza)","(Rice, Cake)",0.125,0.125,0.125,1.000000,8.000000,0.109375,inf,1.000000
...,...,...,...,...,...,...,...,...,...,...
137,(Rice),"(Milk, Salad, Pizza)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
322,(Rice),"(Cake, Candy, Salad)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
451,(Rice),"(Candy, Cake, Pizza, Salad)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
698,(Rice),"(Meal, Soup)",0.875,0.125,0.125,0.142857,1.142857,0.015625,1.020833,1.000000
